In [ ]:
# 모듈 임포트
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np

## 데이터셋 준비

In [ ]:
train_datasets = tfds.load('cassava',split='train')
valid_datasets = tfds.load('cassava',split='validation')
test_datasets = tfds.load('cassava',split='test')

def preprocessing(data):
    image = tf.cast(data['image'],dtype=tf.float32)/255.0
    image = tf.image.resize(image,size=(224,224))
    label = data['label']
    return image,label

train_data = train_datasets.map(preprocessing).shuffle(1000).batch(128)
valid_data = valid_datasets.map(preprocessing).batch(32)
test_data = test_datasets.map(preprocessing).batch(32)

### Transfer Layer 구현

In [ ]:

class Transfer(tf.keras.models.Model):
    def __init__(self):
        super(Transfer,self).__init__()
        self.transfer_model = tf.keras.applications.VGG16(weights = 'imagenet',         # 이미지 분류에 대한 가중치
                                                          include_top = False,          #  fully connected layer 제외
                                                          input_shape = (224,224,3))    # input shape 정의
        self.transfer_model.trainable = False # 기존 VGG16 가중치 고정
        
    
    def call(self,input_):
        x = self.transfer_model(input_)
        return x

### 모델 정의

In [ ]:
class Mymodel(tf.keras.models.Model):
    def __init__(self):
        super(Mymodel,self).__init__()
        self.vgg16 = Transfer()
        self.flatten = tf.keras.layers.Flatten()
        self.Dense = tf.keras.layers.Dense(128,activation='relu')
        self.output_ = tf.keras.layers.Dense(5,activation='softmax')

    def call(self,input_):
        x = self.vgg16(input_)
        x = self.flatten(x)
        x = self.Dense(x)
        x = self.output_(x)
        return x

In [ ]:
model = Mymodel()

input_ = tf.keras.layers.Input(shape=(224,224,3))
model(input_)

model.summary()

Model: "mymodel"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
transfer (Transfer)          multiple                  14714688  
_________________________________________________________________
flatten (Flatten)            multiple                  0         
_________________________________________________________________
dense (Dense)                multiple                  3211392   
_________________________________________________________________
dense_1 (Dense)              multiple                  645       
Total params: 17,926,725
Trainable params: 3,212,037
Non-trainable params: 14,714,688
_________________________________________________________________


## 모델 컴파일,체크포인트,학습

In [ ]:
# 컴파일
optimizer = tf.keras.optimizers.Adam(lr=0.001)
model.compile(optimizer=optimizer,loss='sparse_categorical_crossentropy',metrics=['acc'])

In [ ]:
# 체크 포인트
checkpoint_path = 'my_checkpoint.ckpt'
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                save_weights_only = True,
                                                save_best_only = True,
                                                monitor = 'val_loss',
                                                verbose = 1)

In [ ]:
#학습
model.fit(train_data,
          validation_data=(valid_data),
          epochs=10,
          callbacks = [checkpoint])

Epoch 1/10
45/45 [==============================] - 130s 2s/step - loss: 2.7820 - acc: 0.4393 - val_loss: 0.8774 - val_acc: 0.6617

Epoch 00001: val_loss improved from inf to 0.87744, saving model to my_checkpoint.ckpt
Epoch 2/10
45/45 [==============================] - 78s 2s/step - loss: 0.8178 - acc: 0.6979 - val_loss: 0.8396 - val_acc: 0.6787

Epoch 00002: val_loss improved from 0.87744 to 0.83956, saving model to my_checkpoint.ckpt
Epoch 3/10
45/45 [==============================] - 78s 2s/step - loss: 0.6813 - acc: 0.7549 - val_loss: 0.7714 - val_acc: 0.7094

Epoch 00003: val_loss improved from 0.83956 to 0.77140, saving model to my_checkpoint.ckpt
Epoch 4/10
45/45 [==============================] - 77s 2s/step - loss: 0.5691 - acc: 0.8028 - val_loss: 0.7724 - val_acc: 0.7110

Epoch 00004: val_loss did not improve from 0.77140
Epoch 5/10
45/45 [==============================] - 77s 2s/step - loss: 0.4698 - acc: 0.8527 - val_loss: 0.8262 - val_acc: 0.6871

Epoch 00005: val_loss di

In [ ]:
# 가중치 적용
model.load_weights(checkpoint_path)

In [ ]:
# 테스트
model.evaluate(test_data)

59/59 [==============================] - 23s 395ms/step - loss: 0.7550 - acc: 0.7279


[0.7549511790275574, 0.7278514504432678]